# Working with the Chinook Database

We are going to make some analysis of the Chinook Database to answer interesting/useful question about its data. In summary, this is a Database with data from an online music store. We will briefly explore its data before going into the questions we want to answer. More information about the Database can be found on the Common Databases folder in this repository.


## A First Look at the Data and some useful SQLite commands

Our first step is to connect with the database:

In [1]:
import os
import sqlite3

COMMON_DATABASES_FOLDER = os.path.join(os.path.join(os.path.abspath(".."), "Common_Databases"), "chinook")
CHINOOK_PATH = os.path.join(COMMON_DATABASES_FOLDER, "chinook.db")

conn = sqlite3.connect(CHINOOK_PATH)


**When working with SQLite, we can list all the tables in a Database using a SELECT statement.**

The Chinook Database is composed by 11 tables. In the cell below we see them all, as well as some tables started with "sqlite_", which are automatically generated by SQLite. We only need the original Chinook tables, and so we will filter the results. One should notice that the list returned when calling the fetchall() function is a list of tuple objects, and not a list of strings.

In [2]:
cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")

cursor.fetchall()

[('albums',),
 ('sqlite_sequence',),
 ('artists',),
 ('customers',),
 ('employees',),
 ('genres',),
 ('invoices',),
 ('invoice_items',),
 ('media_types',),
 ('playlists',),
 ('playlist_track',),
 ('tracks',),
 ('sqlite_stat1',)]

Retrieving only the original Chinook tables names can be done in the following way:

In [3]:
tablesNames = [
    table[0] for table in conn.execute('''SELECT name FROM sqlite_master WHERE type='table';''').fetchall() 
    if not "sqlite_" in table[0]
]
tablesNames

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoices',
 'invoice_items',
 'media_types',
 'playlists',
 'playlist_track',
 'tracks']

Since it is a Database designed to study purposes, all the tables in Chinook have small sizes. The greatest one is "playlist_track", with 8715 entries.

In [4]:
for table in tablesNames:
    for item in conn.execute("SELECT COUNT(*) FROM {}".format(table)):
        print("Table \"{}\" size: {}".format(table, item[0]))

Table "albums" size: 347
Table "artists" size: 275
Table "customers" size: 59
Table "employees" size: 8
Table "genres" size: 25
Table "invoices" size: 412
Table "invoice_items" size: 2240
Table "media_types" size: 5
Table "playlists" size: 18
Table "playlist_track" size: 8715
Table "tracks" size: 3503


**Another important information: how to get the Tables columns names/headers**

Besides the tables names, we need to know what each column represents in a Database to really understand what it means and how to extract valuable information from the raw data. When working with SQLite via Python, we can use the "description" attribute of a cursor to retrieve this information.

In [5]:
cursor = conn.execute("SELECT * FROM tracks")
columns = list(map(lambda x: x[0], cursor.description))
columns

['TrackId',
 'Name',
 'AlbumId',
 'MediaTypeId',
 'GenreId',
 'Composer',
 'Milliseconds',
 'Bytes',
 'UnitPrice']

In [6]:
import pandas as pd

pd.read_sql("SELECT * FROM tracks", conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99
